# Cleaning and EDA



In [218]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import stats
import statsmodels.api as sm
from statsmodels.sandbox.regression.predstd import wls_prediction_std

In [219]:
realEstate = pd.read_csv('Ames Real Estate Data.csv')
housePrice = pd.read_csv('Ames_HousePrice.csv', low_memory=False)
re_table = pd.read_csv('RealEstate_Table.csv')

C:\Users\hzeig\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (36,40,81,87) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


### House Price Data

In [220]:
# dictionary of housePrice column meanings for quick reference
with open('housePrice_features') as hp_feat:
    hp_dict = {}
    for line in hp_feat.readlines():
        feature, description = [*line.split(':')]
        hp_dict[feature] = description

In [221]:
hp_dict['LotFrontage']

' Linear feet of street connected to property\n'

In [222]:
# function for viewing columns with missing values
def nas(df):
    return df.loc[:, df.isna().sum() > 0 ].isna().sum().sort_values(ascending = False)

In [223]:
# investigate / impute column NaNs one by one
nas(housePrice)

PoolQC          2571
MiscFeature     2483
Alley           2412
Fence           2055
FireplaceQu     1241
LotFrontage      462
GarageCond       129
GarageQual       129
GarageFinish     129
GarageYrBlt      129
GarageType       127
BsmtExposure      71
BsmtFinType2      70
BsmtFinType1      69
BsmtQual          69
BsmtCond          69
MasVnrArea        14
MasVnrType        14
BsmtHalfBath       2
BsmtFullBath       2
TotalBsmtSF        1
BsmtUnfSF          1
GarageCars         1
GarageArea         1
BsmtFinSF2         1
BsmtFinSF1         1
Electrical         1
dtype: int64

In [224]:
hp_dict['PoolQC']             
housePrice.PoolQC.unique()
housePrice.PoolQC = housePrice.PoolQC.fillna('none')
housePrice.groupby('PoolQC')['SalePrice'].agg(['mean', 'std', 'count'])

,mean,std,count
PoolQC,,,
Ex,465000.000000,212132.034356,2
Fa,215500.000000,48790.367902,2
Gd,215500.000000,62932.503526,2
TA,170500.000000,51529.117982,3
none,177786.980941,74571.704907,2571


In [225]:
hp_dict['MiscFeature']                                                           # drop
housePrice.groupby('MiscFeature')['SalePrice'].agg(['mean', 'std', 'count'])
housePrice = housePrice.drop('MiscFeature', axis = 1)

In [226]:
hp_dict['Alley']
# housePrice.Alley.unique()
housePrice.Alley.value_counts()
housePrice.Alley = housePrice.Alley.fillna('none')
housePrice.groupby('Alley')['SalePrice'].agg(['count','mean','std'])

,count,mean,std
Alley,,,
Grvl,105,126345.990476,36101.160174
Pave,63,178559.698413,45650.146340
none,2412,180297.776949,76096.323125


In [227]:
hp_dict['Fence']  
# Good Privacy - GdPr
# Good Wood - GdWo
# Minimum Privacy - MnPr
# Minimum Wood - MnWw
housePrice.Fence = housePrice.Fence.fillna('none')
housePrice.groupby('Fence')['SalePrice'].agg(['count','mean','std'])

,count,mean,std
Fence,,,
GdPrv,111,176726.576577,62331.695505
GdWo,103,144750.223301,47246.448542
MnPrv,301,146485.810631,44748.238487
MnWw,10,131965.000000,22333.856337
none,2055,184650.134307,78736.302160


In [228]:
hp_dict['FireplaceQu']
housePrice.FireplaceQu = housePrice.FireplaceQu.fillna('none')
housePrice.groupby('FireplaceQu')['SalePrice'].agg(['count','mean','std'])

,count,mean,std
FireplaceQu,,,
Ex,35,340444.542857,112432.168836
Fa,70,168482.142857,36079.397184
Gd,619,223019.756058,91182.521787
Po,43,142528.674419,34116.467478
TA,572,204473.805944,63380.868389
none,1241,140650.717164,41618.136667


In [229]:
hp_dict['LotFrontage']                                 # impute using LotArea
housePrice.loc[housePrice.LotFrontage == 31]
housePrice.groupby('LotFrontage')['SalePrice'].agg(['count','mean','std'])

,count,mean,std
LotFrontage,,,
21.0,46,95106.304348,13631.687979
22.0,1,217500.000000,NaN
24.0,49,144108.163265,20526.251857
25.0,1,260000.000000,NaN
26.0,3,183666.666667,6506.407099
...,...,...,...
168.0,1,274725.000000,NaN
174.0,1,403000.000000,NaN
195.0,1,155000.000000,NaN


#### [Iterative Imputing](https://machinelearningmastery.com/iterative-imputation-for-missing-values-in-machine-learning/)

In [230]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

Lot_df = housePrice[['LotFrontage','LotArea']]
imputer = IterativeImputer()
imputer.fit(Lot_df)
LotTrans = imputer.transform(Lot_df)
Lot_Imputed = pd.DataFrame(LotTrans)
housePrice['LotFrontage'] = Lot_Imputed[0]

In [231]:
nas(housePrice)

GarageCond      129
GarageQual      129
GarageFinish    129
GarageYrBlt     129
GarageType      127
BsmtExposure     71
BsmtFinType2     70
BsmtQual         69
BsmtCond         69
BsmtFinType1     69
MasVnrType       14
MasVnrArea       14
BsmtFullBath      2
BsmtHalfBath      2
Electrical        1
BsmtUnfSF         1
BsmtFinSF2        1
GarageCars        1
GarageArea        1
BsmtFinSF1        1
TotalBsmtSF       1
dtype: int64

In [232]:
hp_dict['GarageCond']                                                       
housePrice.GarageCond = housePrice.GarageCond.fillna('none')
housePrice.groupby('GarageCond')['SalePrice'].agg(['count','mean','std'])

,count,mean,std
GarageCond,,,
Ex,3,125000.000000,3905.124838
Fa,67,108543.791045,37994.751260
Gd,14,188278.571429,72267.145196
Po,11,106618.181818,27559.238675
TA,2356,184277.848048,74495.805841
none,129,106814.604651,33428.251290


In [233]:
hp_dict['GarageQual']                                             # should we try to compile garage? (is that clusters?)
housePrice.GarageQual = housePrice.GarageQual.fillna('none')              # compare relationship between rankings 
housePrice.groupby('GarageQual')['SalePrice'].agg(['count','mean','std'])           # across garage classifications

,count,mean,std
GarageQual,,,
Ex,3,241000.000000,202680.166765
Fa,110,118884.100000,39721.189121
Gd,22,242833.500000,119708.930561
Po,4,94350.000000,31556.140448
TA,2312,184297.039792,73708.905056
none,129,106814.604651,33428.251290


In [234]:
hp_dict['GarageFinish']                                             
housePrice.GarageFinish = housePrice.GarageFinish.fillna('none')
housePrice.groupby('GarageFinish')['SalePrice'].agg(['count','mean','std'])

,count,mean,std
GarageFinish,,,
Fin,621,231925.247987,89751.688867
RFn,718,199724.565460,64714.442479
Unf,1112,142254.430755,44088.459471
none,129,106814.604651,33428.251290


In [235]:
hp_dict['GarageYrBlt']                                                         
housePrice.GarageYrBlt = housePrice.GarageYrBlt.fillna('none')
housePrice.groupby('GarageYrBlt')['SalePrice'].agg(['count','mean','std'])

,count,mean,std
GarageYrBlt,,,
1895.0,1,89000.000000,NaN
1900.0,5,124760.000000,31569.494769
1906.0,1,135000.000000,NaN
1908.0,1,240000.000000,NaN
1910.0,10,113530.000000,29867.226557
...,...,...,...
2007.0,67,258978.552239,98539.246379
2008.0,44,292216.795455,87551.849537
2009.0,12,297876.666667,126085.057796


In [236]:
hp_dict['GarageType']                                             
housePrice.GarageType = housePrice.GarageType.fillna('none')
housePrice.groupby('GarageType')['SalePrice'].agg(['count','mean','std'])

,count,mean,std
GarageType,,,
2Types,21,154080.952381,36478.125213
Attchd,1527,200661.569090,75910.133901
Basment,27,155150.000000,33391.342153
BuiltIn,153,233054.464052,80768.842199
CarPort,9,110716.666667,27859.446333
Detchd,716,133291.765363,39764.381342
none,127,106048.622047,33116.655928


In [237]:
hp_dict['BsmtExposure']                                             
housePrice.BsmtExposure = housePrice.BsmtExposure.fillna('none')
housePrice.groupby('BsmtExposure')['SalePrice'].agg(['count','mean','std'])

,count,mean,std
BsmtExposure,,,
Av,344,201482.340116,76537.790904
Gd,242,262873.061983,113165.344846
Mn,215,185463.330233,70096.651361
No,1708,163135.738876,57349.711224
none,71,112086.816901,42548.995226


In [238]:
hp_dict['BsmtFinType2']                                             
housePrice.BsmtFinType2 = housePrice.BsmtFinType2.fillna('none')
housePrice.groupby('BsmtFinType2')['SalePrice'].agg(['count','mean','std'])

,count,mean,std
BsmtFinType2,,,
ALQ,53,188644.339623,82192.691433
BLQ,64,161466.140625,57560.260186
GLQ,32,211278.125000,101942.759031
LwQ,84,160725.297619,38958.990930
Rec,97,164338.402062,55684.315058
Unf,2180,181206.894954,76175.880106
none,70,111830.914286,44695.624964


In [239]:
hp_dict['BsmtFinType1']                                             
housePrice.BsmtFinType1 = housePrice.BsmtFinType1.fillna('none')
housePrice.groupby('BsmtFinType1')['SalePrice'].agg(['count','mean','std'])

,count,mean,std
BsmtFinType1,,,
ALQ,397,163515.198992,48331.383704
BLQ,251,150852.749004,44819.815866
GLQ,753,231889.369190,87608.901286
LwQ,139,153106.834532,55352.198810
Rec,265,144782.030189,49548.992468
Unf,706,162617.998584,63191.379338
none,69,109335.710145,39810.106680


In [240]:
hp_dict['BsmtQual']                                            # does it matter if some basements (garages) don't have           
housePrice.BsmtQual = housePrice.BsmtQual.fillna('none')       # certain classifications, and are seen as 'none'
housePrice.groupby('BsmtQual')['SalePrice'].agg(['count','mean','std'])

,count,mean,std
BsmtQual,,,
Ex,188,327308.005319,102928.298214
Fa,78,113467.051282,38699.187921
Gd,1077,200961.085422,56495.645593
Po,2,84950.000000,5727.564928
TA,1166,141389.656947,40290.731298
none,69,109335.710145,39810.106680


In [241]:
hp_dict['BsmtCond']                                             
housePrice.BsmtCond = housePrice.BsmtCond.fillna('none')
housePrice.groupby('BsmtCond')['SalePrice'].agg(['count','mean','std'])

,count,mean,std
BsmtCond,,,
Ex,3,195000.000000,83216.584885
Fa,88,122382.875000,44683.700374
Gd,101,206709.128713,69427.407860
Po,3,95133.333333,34428.706530
TA,2316,181058.699914,74981.038735
none,69,109335.710145,39810.106680


In [242]:
hp_dict['MasVnrType']
housePrice.MasVnrType = housePrice.MasVnrType.fillna('unknown')
housePrice.groupby('MasVnrType')['SalePrice'].agg(['count','mean','std'])

,count,mean,std
MasVnrType,,,
BrkCmn,20,145450.200000,43092.544242
BrkFace,804,208615.353234,82393.220408
None,1559,154365.317511,54210.546901
Stone,183,246845.519126,101989.391799
unknown,14,209272.857143,51066.242866


In [243]:
# iterative imputer again, using area-related columns
hp_dict['MasVnrArea']
size_related = housePrice.filter(regex='SF|Area$')

impute = IterativeImputer(max_iter=100)
size_trans = impute.fit_transform(size_related)
size_trans = pd.DataFrame(size_trans)
housePrice.MasVnrArea = size_trans[2]
housePrice.BsmtFinSF1 = size_trans[3]
housePrice.BsmtFinSF2 = size_trans[4]
housePrice.BsmtUnfSF = size_trans[5]
housePrice.TotalBsmtSF = size_trans[6]
housePrice.GarageArea = size_trans[8]

C:\Users\hzeig\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:685: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn("[IterativeImputer] Early stopping criterion not"


In [244]:
list(enumerate(size_related))

[(0, 'GrLivArea'),
 (1, 'LotArea'),
 (2, 'MasVnrArea'),
 (3, 'BsmtFinSF1'),
 (4, 'BsmtFinSF2'),
 (5, 'BsmtUnfSF'),
 (6, 'TotalBsmtSF'),
 (7, '1stFlrSF'),
 (8, '2ndFlrSF'),
 (9, 'LowQualFinSF'),
 (10, 'GarageArea'),
 (11, 'WoodDeckSF'),
 (12, 'OpenPorchSF'),
 (13, 'PoolArea')]

In [245]:
nas(housePrice)

BsmtFullBath    2
BsmtHalfBath    2
Electrical      1
GarageCars      1
dtype: int64

In [246]:
hp_dict['BsmtHalfBath']
housePrice.groupby('BsmtFullBath')['SalePrice'].agg(['count','mean','std'])
housePrice[housePrice['BsmtFullBath'].isna()]

bath_df = housePrice.filter(regex='Bath$')

impute = IterativeImputer(max_iter=100)
bath_trans = impute.fit_transform(bath_df)
bath_trans = pd.DataFrame(bath_trans)
housePrice.BsmtFullBath = bath_trans[0]
housePrice.BsmtHalfBath = bath_trans[1]

In [247]:
list(enumerate(bath_df))

[(0, 'BsmtFullBath'), (1, 'BsmtHalfBath'), (2, 'FullBath'), (3, 'HalfBath')]

In [248]:
nas(housePrice)

Electrical    1
GarageCars    1
dtype: int64

In [249]:
hp_dict['Electrical']
housePrice[housePrice['Electrical'].isna()]
housePrice.groupby('Electrical')['SalePrice'].agg(['count','mean','std'])
housePrice['Electrical'] = housePrice['Electrical'].fillna('SBrkr')

In [250]:
hp_dict['GarageCars']
housePrice[housePrice['GarageCars'].isna()]
housePrice.groupby('GarageCars')['SalePrice'].agg(['count','mean','std'])

garage = housePrice[['GarageCars', 'GarageArea']]

impute = IterativeImputer(max_iter=100)
garage_trans = impute.fit_transform(garage)
garage_trans = pd.DataFrame(garage_trans)
housePrice.GarageCars = garage_trans[0]

In [253]:
nas(housePrice)

Series([], dtype: float64)

### Real Estate Data

In [251]:
realEstate.shape

(22213, 91)

In [252]:
nas(realEstate)

MA_Line2     22213
PA-PostD     22213
X2TPr_D      22212
X2TSc_S      22212
X2TPr_S      22212
             ...  
Prop_Addr       20
MA_Zip1         14
MA_State        14
MA_City         14
MA_Line1        14
Length: 65, dtype: int64

""
0
1
2
3
4
...
22208
22209
22210
22211
